Os tempos de execução de duas cargas de trabalho foram medidos em dois processadores diferentes,           
A e B. Cada experimento foi repetido 3 vezes.

| Carga | Processador A | Processador B |
|-------|---------------|---------------|
| 1     | 41,39,42      | 63,59,64      |
| 2     | 51,52,50      | 48,49,47      |

#### Você pode dizer, com 90% de confiança, que os tempos de execução da carga 2 são diferentes nos dois processadores? Você pode dizer, com 90% de confiança, que a carga 2 executa mais rápido no  processador B? Qual a maior confiança que você pode atribuir a estas duas afirmativas?

In [9]:
# Para responder a primeira pergunta, fazemos um teste t não-pareado.
import numpy as np

carga2ProcA = [51, 52, 50]
carga2ProcB = [48, 49, 47]

mediaA = np.mean(carga2ProcA)
mediaB = np.mean(carga2ProcB)
print(f"Media A = {mediaA:.4f}")
print(f"Media B = {mediaB:.4f}")

diferencaMedia = mediaA - mediaB
print(f"Diferença das Médias: {diferencaMedia:.4f}")

Sa = np.std(carga2ProcA, ddof=1)
Sb = np.std(carga2ProcB, ddof=1)
print(f"Sa = {Sa:.4f}")
print(f"Sb = {Sb:.4f}")

# Para facilitar a conta do S e dos graus de liberdade
Sa2_sub_Na = (np.std(carga2ProcA, ddof=1) ** 2) / len(carga2ProcA)
Sb2_sub_Nb = (np.std(carga2ProcB, ddof=1) ** 2) / len(carga2ProcB)

S = np.sqrt(Sa2_sub_Na + Sb2_sub_Nb)
print(f"S: {S:.4f}")

# Calculando graus de liberdade
dof = (S ** 4) / ( (1 / (len(carga2ProcA)-1) * (Sa2_sub_Na**2) ) + ((1 / (len(carga2ProcB)-1) * (Sb2_sub_Nb**2) )) ) - 2
print(f"Graus de Liberdade: {dof:.4f}")

# Agora que temos todos os dados, vamos ver se 
# o IC da diferença das médias e do desvio padrão inclui o 0.

# https://www.ttable.org/

valorTabelaT = 2.920
lowerBound = 3 - 0.8165 * valorTabelaT
upperBound = 3 + 0.8165 * valorTabelaT

print(f"IC com 90% de confiança diferentes = [{lowerBound}, {upperBound}]")


valorTabelaT = 1.886
lowerBound = 3 - 0.8165 * valorTabelaT
upperBound = 3 + 0.8165 * valorTabelaT

print(f"IC com 90% de confiança superior = [{lowerBound}, {upperBound}]")

Media A = 51.0000
Media B = 48.0000
Diferença das Médias: 3.0000
Sa = 1.0000
Sb = 1.0000
S: 0.8165
Graus de Liberdade: 2.0000
IC com 90% de confiança diferentes = [0.6158200000000003, 5.38418]
IC com 90% de confiança superior = [1.4600810000000002, 4.539918999999999]


#### Considerando um projeto 2kr com dois fatores (a carga e o processador), quais são as estimativas e desvios padrões para cada efeito? Apresente um modelo para previsão do tempo de processamento dados carga e processador.

Como estamos medindo o tempo e tempo é tempo de processamento / carga * carga, o modelo é multiplicativo. Sempre que o modelo é multiplicativo, tiramos log das medições.

In [13]:
import pandas as pd
import numpy as np

tabela = pd.DataFrame(
                    [
                        [1, -1, -1,  1, 30.58, 30.49, 30.44],
                        [1,  1, -1, -1, 30.04, 31.44, 31.15],
                        [1, -1,  1, -1, 30.03, 29.12, 30.77],
                        [1,  1,  1,  1, 29.19, 30.63, 29.90]
                    ],
                    columns=["0", "Step Size", "nFrames", "Interacao", "y1", "y2", "y3"]
                )

#for col in ["y1", "y2", "y3"]:
#    logValues = [np.emath.log10(i) for i in tabela[col]]
#    tabela[f"{col}_log"] = logValues

medias = []
for i in range(4):
    media = 0
    for col in ["y1", "y2", "y3"]:
        media += tabela[col][i]
    media = media / 3

    medias.append(media)

tabela["y_medio"] = medias
#print(tabela)


# Divido por 4 porque são 4 niveis na tabela
q0 = np.sum(tabela["y_medio"]) / 4

total = 0
for cNivel, yMed in zip(tabela["Step Size"], tabela["y_medio"]):
    total += cNivel * yMed
qA = total / 4

total = 0
for cNivel, yMed in zip(tabela["nFrames"], tabela["y_medio"]):
    total += cNivel * yMed
qB = total / 4

total = 0
for cNivel, yMed in zip(tabela["Interacao"], tabela["y_medio"]):
    total += cNivel * yMed
qAB = total / 4

print(f"q0 = {q0:.4f}, qA = {qA:.4f}, qB = {qB:.4f}, qAB = {qAB:.4f}")

# Quando chegamos nesse ponto de saber quem são os q0, q1, etc, podemos montar o nosso modelo com eles.

# Modelo linear:
# Log(t) = 1.6975 -0.0031Xa + 0.0392Xb - 0.0524XaXb



q0 = 30.3150, qA = 0.0767, qB = -0.3750, qAB = -0.1100


#### Qual a porcentagem de variação explicada por cada efeito? Você está satisfeito com o  seu modelo? Justifique

Para saber a porcentagem de variação explicada, temos que calcular o SSA, SSB e SSAB. 
Como já sabemos o qA, qB e qAB é muito facil.

In [14]:
nReplicacoes = 3
SSA  = 4 * nReplicacoes * qA**2
SSB  = 4 * nReplicacoes * qB**2
SSAB = 4 * nReplicacoes * qAB**2

# Em um projeto com replicação, a fórmula do SST muda!
# Em um projeto 2k sem replicacão, a fórmula do SST é:
# SST = 4 * qA**2 + 4 * qB**2 + 4 * qAB**2;

# Mas quando tem replicação, a fórmula muda para:
# Soma todos os y obtidos (ou logY, nesse caso como convertemos para log)
# e depois subtrai desse valor 4 * nReplicacoes * q0^2
total = 0
a = list(tabela["y1"])
a.extend(tabela["y2"])
a.extend(tabela["y3"])
for i in a:
    total += i **2

SST = total - (4 * nReplicacoes * q0**2)
print(f"SST: {SST}")

# Esse total que calculamos é o SSY no projeto 2^kr
SSY = total
print(f"SSY: {SSY}")

# E o (4 * nReplicacoes * q0**2) é o SS0
SS0 = (4 * nReplicacoes * q0**2)
print(f"SS0: {SS0}")

# O SSE é o SSY - SS0 - SSA - SSB - SSAB
SSE = SSY - SS0 - SSA - SSB - SSAB
print(f"SSE: {SSE}")

# Para descobrir quanto da variação cada um explica, dividimos o SS que queremos pelo SST
variacaoSSA  = 100 * SSA/SST
variacaoSSB  = 100 * SSB/SST
variacaoSSAB = 100 * SSAB/SST
variacaoSSE  = 100 * SSE/SST

print(f"Variacao explicada pelo SSA: {variacaoSSA:.4f}")
print(f"Variacao explicada pelo SSB: {variacaoSSB:.4f}")
print(f"Variacao explicada pelo SSAB: {variacaoSSAB:.4f}")
print(f"Variacao explicada pelo SSE: {variacaoSSE:.4f}")

SST: 5.408300000002782
SSY: 11033.399000000001
SS0: 11027.990699999998
SSE: 3.505066666669455
Variacao explicada pelo SSA: 1.3042
Variacao explicada pelo SSB: 31.2020
Variacao explicada pelo SSAB: 2.6848
Variacao explicada pelo SSE: 64.8090


#### Com confiança de 90%, qual o tempo de execução estimado para a média de 10 execuções futuras da  carga 1 no processador B? E para apenas uma execução futura na mesma configuração? Compare as  precisões destas duas estimativas.

In [119]:
# Jogando no nosso modelo, logT é o valor previsto pelo modelo.
logT = 1.6975 -0.0031 * -1 + 0.0392 * 1 - 0.0524 * -1
print(f"Valor Predito (escala log): {logT:.4f}")

# O Se não é o desvio padrão dos efeitos, mas é usado para calcular eles. 
# Aqui no caso não vamos precisar
Se = np.sqrt(SSE / (4*(nReplicacoes-1)) )
print(f"Se: {Se:.4f}")

# Estimativa de resposta para M experimentos no futuro:
M = 1
S = Se * np.sqrt( (5 / (4*nReplicacoes)) + (1/M) )
print(f"S: {S:.4f}")

# Agora, calculando o IC para a estimativa de resposta e o desvio padrão das respostas.
# Graus de Liberdade = 2^2 * (nreplicacoes-1)

dof = 4 * (nReplicacoes-1)
print(f"Graus de Liberdade: {dof:.4f}")

# Agora vamos na tabela T olhar qual T usar!
# https://www.ttable.org/

# Aqui usamos um T two-tailed 90% de confiança para 8 graus de liberdade, que é 1.860
valorTabelaT = 1.860

lowerBound   = logT - valorTabelaT * S
upperBound   = logT + valorTabelaT * S

print(f"IC Calculado para {M} execucoes futuras: [{lowerBound:.4f}, {upperBound:.4f}]")

Valor Predito (escala log): 1.7922
Se: 0.0139
S: 0.0166
Graus de Liberdade: 8.0000
IC Calculado para 1 execucoes futuras: [1.7614, 1.8230]
